In [4]:
import dspy
from src.roma_dspy.modules import Executor, Atomizer, Planner, Aggregator
from src.roma_dspy.engine.solve import RecursiveSolver
from src.roma_dspy.engine.visualizer import ExecutionVisualizer
from src.roma_dspy.engine.solve import event_solve, solve

executor_lm = dspy.LM("openrouter/google/gemini-2.5-flash", cache=False)
atomizer_lm = dspy.LM("openrouter/google/gemini-2.5-flash", cache=False)
planner_lm = dspy.LM("openrouter/google/gemini-2.5-flash", cache=False)
aggregator_lm = dspy.LM("openrouter/google/gemini-2.5-flash", cache=False)

# Initialize modules
atomizer = Atomizer(lm=atomizer_lm)
planner = Planner(lm=planner_lm)
executor = Executor(lm=executor_lm)
aggregator = Aggregator(lm=aggregator_lm)

# Create solver
solver = RecursiveSolver(
    atomizer,
    planner,
    executor,
    aggregator,
    max_depth=2
)

In [ ]:
# Test task - use a simpler task for clearer output
task_goal = "Write me a blog post about the benefits of using DSPy."

print("=" * 80)
print("ROMA-DSPy Detailed Execution Visualization")
print("=" * 80)
print(f"\n🎯 Goal: {task_goal}\n")

# Execute task
print("⚙️  Executing task...\n")
result = solver.solve(task_goal)

In [ ]:
# Get the DAG for detailed visualization
dag = solver.last_dag

if dag:
    # Show the full execution report
    visualizer = ExecutionVisualizer()
    print(visualizer.get_full_execution_report(result, dag))

In [ ]:
print(visualizer.get_execution_tree_with_details(result, dag))

In [ ]:
print(result.result)

In [ ]:
# Use async/await pattern - most compatible with Jupyter
import asyncio

async def run_task():
    task_goal = "Write me a blog post about the benefits of using DSPy."

    print("=" * 80)
    print("ROMA-DSPy Detailed Execution Visualization")
    print("=" * 80)
    print(f"\n🎯 Goal: {task_goal}\n")

    # Execute task
    print("⚙️  Executing task...\n")
    result = await solver.async_event_solve(task_goal, concurrency=8)

    print(f"\n✅ Result: {result.result}")
    return result

# Run it
new_result = await run_task()

ROMA-DSPy Detailed Execution Visualization

🎯 Goal: Write me a blog post about the benefits of using DSPy.

⚙️  Executing task...


✅ Result: # Unlocking LLM Potential: Why DSPy is a Game-Changer for AI Development

## I. Introduction

Are you tired of wrestling with complex prompt engineering, struggling to get consistent and reliable outputs from your Large Language Models (LLMs)? What if there was a way to program LLMs more systematically, like traditional software, and achieve significantly better results with less effort?

Enter DSPy, a groundbreaking framework designed to optimize and program LLMs. Unlike traditional prompt engineering, which often relies on trial-and-error, DSPy treats LLMs as components within a larger program, allowing developers to compose, compile, and optimize complex LLM pipelines. It provides a systematic approach to building robust and efficient LLM-powered applications, moving beyond simple prompting to a more structured and performant paradigm.

In an 